In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import torch

from dystformer.patchtst.pipeline import PatchTSTPipeline
from dystformer.utils import (
    apply_custom_style,
    get_system_filepaths,
    load_trajectory_from_arrow,
)

In [ ]:
# Apply matplotlib style from config
apply_custom_style("../config/plotting.yaml")

In [ ]:
run_name = "mlm_stand_chattn_noembed-0"  # NOTE: this is still the best
model = PatchTSTPipeline.from_pretrained(
    mode="pretrain",
    pretrain_path=f"/stor/work/AMDG_Gilpin_Summer2024/checkpoints/{run_name}/checkpoint-final",
    device_map="cuda:6",
)

In [ ]:
model.model.model.encoder.layers[0]

In [ ]:
# def plot_model_completion(
#     model,
#     context: np.ndarray,
#     figsize: tuple[int, int] = (6, 8),
#     return_normalized_completions: bool = False,
# ):
#     # Prepare input tensor
#     context_tensor = torch.from_numpy(context.T).float().to(model.device)[None, ...]
#     print(f"context_tensor shape: {context_tensor.shape}")

#     # Generate completions
#     completions_output = model.model.generate_completions(
#         context_tensor, past_observed_mask=None
#     )
#     print(f"completions output shape: {completions_output.completions.shape}")

#     # Extract shapes and data
#     patch_size = completions_output.completions.shape[-1]

#     # Check for required outputs
#     if any(x is None for x in [completions_output.mask, completions_output.patched_past_values]):
#         raise ValueError("Required completion outputs are None")

#     # Process tensors to numpy arrays
#     def process_tensor(tensor, reshape=True):
#         if reshape:
#             return (tensor.reshape(context_tensor.shape[0], context_tensor.shape[-1], -1)
#                    .detach().cpu().numpy().transpose(0, 2, 1))
#         return tensor.detach().cpu().numpy()

#     completions = process_tensor(completions_output.completions)
#     processed_context = process_tensor(completions_output.patched_past_values)
#     patch_mask = process_tensor(completions_output.mask, reshape=False)
#     timestep_mask = np.repeat(patch_mask, repeats=patch_size, axis=2)

#     # Denormalize if needed
#     if not return_normalized_completions:
#         if completions_output.loc is None or completions_output.scale is None:
#             raise ValueError("Loc or scale is None")
#         loc = completions_output.loc.detach().cpu().numpy()
#         scale = completions_output.scale.detach().cpu().numpy()
#         completions = completions * scale + loc
#         processed_context = processed_context * scale + loc

#     # Reshape for plotting
#     processed_context = processed_context.squeeze(0).transpose(1, 0)
#     completions = completions.squeeze(0).transpose(1, 0)
#     timestep_mask = timestep_mask.squeeze(0)

#     print(f"processed context shape: {processed_context.shape}")
#     print(f"completions shape: {completions.shape}")
#     print(f"timestep mask shape: {timestep_mask.shape}")

#     n_timesteps = processed_context.shape[1]
#     assert n_timesteps == completions.shape[1] == processed_context.shape[1]

#     # Create figure with grid layout
#     fig = plt.figure(figsize=figsize)
#     gs = gridspec.GridSpec(4, 2, height_ratios=[2, 1, 1, 1])

#     # Create axes
#     ax1 = fig.add_subplot(gs[0, 0], projection="3d")
#     ax2 = fig.add_subplot(gs[0, 1], projection="3d")
#     axes_2d = [fig.add_subplot(gs[i, :]) for i in range(1, 4)]

#     # Plot context in 3D
#     for ax, title, alpha in [(ax1, "Context", 0.8), (ax2, "Completions", 0.3)]:
#         ax.plot(
#             processed_context[0, :],
#             processed_context[1, :],
#             processed_context[2, :],
#             alpha=alpha,
#             color="black",
#             linewidth=1,
#         )
#         ax.set_title(title, y=0.94, fontweight="bold")
#         ax.axis("off")
#         ax.grid(False)

#     # Plot completions in 3D
#     mask_bool = timestep_mask.astype(bool)
#     for dim in range(3):
#         # Find contiguous blocks in mask
#         change_indices = np.where(np.diff(np.concatenate(([False], mask_bool[dim], [False]))))[0]

#         # Plot each contiguous block
#         for i in range(0, len(change_indices), 2):
#             if i + 1 < len(change_indices):
#                 start_idx, end_idx = change_indices[i], change_indices[i + 1]
#                 # Plot masked parts in red
#                 ax2.plot(
#                     completions[0, start_idx:end_idx],
#                     completions[1, start_idx:end_idx],
#                     completions[2, start_idx:end_idx],
#                     alpha=1,
#                     color="red",
#                     linewidth=1,
#                     zorder=10
#                 )
#                 # Plot masked parts in red
#                 ax2.plot(
#                     processed_context[0, start_idx:end_idx],
#                     processed_context[1, start_idx:end_idx],
#                     processed_context[2, start_idx:end_idx],
#                     alpha=1,
#                     color="black",
#                     linewidth=1,
#                 )

#         # # Plot unmasked parts in black
#         # for i in range(1, len(change_indices), 2):
#         #     if i + 1 < len(change_indices):
#         #         start_idx, end_idx = change_indices[i], change_indices[i + 1]
#         #         ax2.plot(
#         #             completions[0, start_idx:end_idx],
#         #             completions[1, start_idx:end_idx],
#         #             completions[2, start_idx:end_idx],
#         #             alpha=0.8,
#         #             color="black",
#         #             linewidth=1,
#         #         )

#     # Plot univariate series for each dimension
#     for dim, ax in enumerate(axes_2d):
#         mask_bool_dim = timestep_mask[dim, :].astype(bool)

#         # Plot context
#         ax.plot(processed_context[dim, :], alpha=0.5, color="black", linewidth=2)

#         # Find segments where mask changes
#         diffs = np.diff(mask_bool_dim.astype(int))
#         change_indices = np.where(diffs)[0]
#         if not mask_bool_dim[0]:
#             change_indices = np.concatenate(([0], change_indices))
#         segment_indices = np.concatenate((change_indices, [n_timesteps]))

#         # Plot completions for masked segments
#         segments = zip(segment_indices[:-1], segment_indices[1:])
#         masked_segments = [idx for i, idx in enumerate(segments) if (i + 1) % 2 == 1]
#         for start, end in masked_segments:
#             if end < n_timesteps - 1:
#                 end += 1
#             ax.plot(
#                 range(start, end),
#                 completions[dim, start:end],
#                 alpha=1,
#                 color="red",
#                 linewidth=2,
#             )

#         # Fill between completions and context
#         ax.fill_between(
#             range(n_timesteps),
#             processed_context[dim, :],
#             completions[dim, :],
#             where=~mask_bool_dim,
#             alpha=0.2,
#         )
#         ax.set_xticks([])
#         ax.set_yticks([])

#     plt.tight_layout()
#     plt.show()

In [ ]:
def plot_model_completion(
    model,
    context: np.ndarray,
    figsize: tuple[int, int] = (6, 8),
    return_normalized_completions: bool = False,
):
    # Prepare input tensor
    context_tensor = torch.from_numpy(context.T).float().to(model.device)[None, ...]
    print(f"context_tensor shape: {context_tensor.shape}")

    # Generate completions
    completions_output = model.model.generate_completions(
        context_tensor, past_observed_mask=None
    )
    print(f"completions output shape: {completions_output.completions.shape}")

    # Extract shapes and data
    patch_size = completions_output.completions.shape[-1]

    # Check for required outputs
    if any(
        x is None
        for x in [completions_output.mask, completions_output.patched_past_values]
    ):
        raise ValueError("Required completion outputs are None")

    # Process tensors to numpy arrays
    def process_tensor(tensor, reshape=True):
        if reshape:
            return (
                tensor.reshape(context_tensor.shape[0], context_tensor.shape[-1], -1)
                .detach()
                .cpu()
                .numpy()
                .transpose(0, 2, 1)
            )
        return tensor.detach().cpu().numpy()

    completions = process_tensor(completions_output.completions)
    processed_context = process_tensor(completions_output.patched_past_values)
    patch_mask = process_tensor(completions_output.mask, reshape=False)
    timestep_mask = np.repeat(patch_mask, repeats=patch_size, axis=2)

    # Denormalize if needed
    if not return_normalized_completions:
        if completions_output.loc is None or completions_output.scale is None:
            raise ValueError("Loc or scale is None")
        loc = completions_output.loc.detach().cpu().numpy()
        scale = completions_output.scale.detach().cpu().numpy()
        completions = completions * scale + loc
        processed_context = processed_context * scale + loc

    # Reshape for plotting
    processed_context = processed_context.squeeze(0).transpose(1, 0)
    completions = completions.squeeze(0).transpose(1, 0)
    timestep_mask = timestep_mask.squeeze(0)

    print(f"processed context shape: {processed_context.shape}")
    print(f"completions shape: {completions.shape}")
    print(f"timestep mask shape: {timestep_mask.shape}")

    n_timesteps = processed_context.shape[1]
    assert n_timesteps == completions.shape[1] == processed_context.shape[1]

    # Create figure with grid layout
    fig = plt.figure(figsize=figsize)
    gs = gridspec.GridSpec(4, 1, height_ratios=[3, 1, 1, 1])

    # Create axes
    ax_3d = fig.add_subplot(gs[0], projection="3d")
    axes_2d = [fig.add_subplot(gs[i]) for i in range(1, 4)]

    # Plot completions in 3D
    ax_3d.plot(
        processed_context[0, :],
        processed_context[1, :],
        processed_context[2, :],
        alpha=0.5,
        color="black",
        linewidth=2,
    )
    # ax_3d.set_title("Completions", y=0.94, fontweight="bold")
    ax_3d.axis("off")
    ax_3d.grid(False)

    # Plot masked segments in 3D
    mask_bool = timestep_mask.astype(bool)
    for dim in range(3):
        # Find contiguous blocks in mask
        change_indices = np.where(
            np.diff(np.concatenate(([False], mask_bool[dim], [False])))
        )[0]

        # Plot each contiguous block
        for i in range(0, len(change_indices), 2):
            if i + 1 < len(change_indices):
                start_idx, end_idx = change_indices[i], change_indices[i + 1]
                # Plot masked parts in red
                ax_3d.plot(
                    completions[0, start_idx:end_idx],
                    completions[1, start_idx:end_idx],
                    completions[2, start_idx:end_idx],
                    alpha=1,
                    color="red",
                    linewidth=2,
                    zorder=10,
                )
                # Plot masked parts in red
                ax_3d.plot(
                    processed_context[0, start_idx:end_idx],
                    processed_context[1, start_idx:end_idx],
                    processed_context[2, start_idx:end_idx],
                    alpha=1,
                    color="black",
                    linewidth=2,
                )

    # Plot univariate series for each dimension
    for dim, ax in enumerate(axes_2d):
        mask_bool_dim = timestep_mask[dim, :].astype(bool)

        # Plot context
        ax.plot(processed_context[dim, :], alpha=0.5, color="black", linewidth=2)

        # Find segments where mask changes
        diffs = np.diff(mask_bool_dim.astype(int))
        change_indices = np.where(diffs)[0]
        if not mask_bool_dim[0]:
            change_indices = np.concatenate(([0], change_indices))
        segment_indices = np.concatenate((change_indices, [n_timesteps]))

        # Plot completions for masked segments
        segments = zip(segment_indices[:-1], segment_indices[1:])
        masked_segments = [idx for i, idx in enumerate(segments) if (i + 1) % 2 == 1]
        for start, end in masked_segments:
            if end < n_timesteps - 1:
                end += 1
            ax.plot(
                range(start, end),
                completions[dim, start:end],
                alpha=1,
                color="red",
                linewidth=2,
                zorder=10,
            )
            ax.plot(
                range(start, end),
                processed_context[dim, start:end],
                alpha=1,
                color="black",
                linewidth=2,
            )

        # Fill between completions and context
        ax.fill_between(
            range(n_timesteps),
            processed_context[dim, :],
            completions[dim, :],
            where=~mask_bool_dim,
            alpha=0.2,
        )
        ax.set_xticks([])
        ax.set_yticks([])

    plt.tight_layout()
    plt.show()

In [ ]:
split = "final_skew40"
test_data_dir = f"/stor/work/AMDG_Gilpin_Summer2024/data/improved/{split}"
subsplit = "test_zeroshot"

In [ ]:
test_system_subdirs = os.listdir(os.path.join(test_data_dir, subsplit))
test_system_subdirs = [
    d
    for d in test_system_subdirs
    if os.path.isdir(os.path.join(test_data_dir, subsplit, d))
]
print(len(test_system_subdirs))

In [ ]:
chosen_forecast_settings = {
    # "KawczynskiStrizhak_HyperXu": (1, 1260, 128, 1),
    # "ForcedBrusselator_SprottB": (2, 770, 128, 1),
    # "SaltonSea_Duffing": (0, 1999, 128, 1),
    # "Bouali_Laser": (10, 55, 128, 1),
    # "QiChen": (0, 980, 256, 1),
    # "Lorenz96_SprottB": (10, 1999, 512, 1),
    # "LorenzStenflo_IsothermalChemical": (0, 322, 256, 1),
    "Coullet_GenesioTesi": (0, 300, 128, 1),
    # "SprottK_Duffing": (6, 815, 256, 1),
}

In [ ]:
print(len(chosen_forecast_settings.keys()))

In [ ]:
context_length = 512

for dyst_name, settings in chosen_forecast_settings.items():
    sample_idx, start_time, pred_length, subsample_interval = settings

    syspaths = get_system_filepaths(dyst_name, test_data_dir, subsplit)
    trajectory, _ = load_trajectory_from_arrow(syspaths[sample_idx])
    trajectory = trajectory[:, ::subsample_interval]

    end_time = start_time + context_length

    save_path = os.path.join(
        "../figures",
        run_name,
        split,
        subsplit,
        f"{dyst_name}_sample{sample_idx}_context{start_time}-{end_time}_pred{pred_length}_.pdf",
    )

    plot_model_completion(
        model,
        trajectory[:, start_time:end_time],  # context
        return_normalized_completions=False,
    )